In [28]:
#using spaCy's english model to do cleaning
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

#creating tokens
def tokenize(text):
    #listning them together
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            #lowercasing tokens for consistency
            lda_tokens.append(token.lower_)
    return lda_tokens

In [29]:
import nltk
from nltk.corpus import wordnet
def get_lem(word):
    lem = wordnet.morphy(word)
    if lem is None:
        return word
    else:
        return lem

In [30]:
#using english stop-words from nltk
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

#function to tokenize + clean
def prepare_for_lda(text):
    #tokenize
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    #remove stop-words
    tokens = [token for token in tokens if token not in en_stop]
    #lemmatize
    tokens = [get_lem(token) for token in tokens]
    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Eshita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
import random
text_data = []
#load in csv--this one using the articles2100 data
with open('articles2100.csv') as f:
    for line in f:
        tokens = prepare_for_lda(line)
        #comment off below if you want to receive tokens for all documents(more accurate but slower)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['another', 'reason', 'people', 'still', 'without', 'health', 'insurance', 'simply', 'afford', 'kaiser', 'family', 'foundation', 'percent', 'remain', 'uninsured', 'qualify', 'credits', 'exchange', 'decline', 'health', 'benefit', 'offer', 'employer', 'another', 'percent', 'uninsured', 'subsidy', 'marketplace']
[]
['relate', 'coverage']
[]
[]
[]
['tracy', 'would', 'accurate', 'get', 'nicely', 'angry', 'although', 'allow', 'might', 'curse', 'twice']
[]
['share', 'gardner', 'ministry', 'great', 'length', 'ensure', 'prospective', 'member', 'understand', 'share', 'insurance', 'guideline', 'spell', 'limit', 'member', 'indicate', 'separate', 'signature', 'understand', 'limit', 'expert', 'consumer', 'advocate', 'worry', 'people', 'ministry', 'realize', 'implication', 'limit']
['affordable', 'insurance', 'already', 'really', 'expensive', 'others', 'qualify', 'credits', 'polafox', 'insurance', 'reform', 'effect', 'price', 'climb', 'since', 'switching', 'higher', 'ded

['carrie', 'banahan', 'executive', 'director', 'kynect']
[]
[]
[]
['immediately', 'trump', 'administration', 'ensure', 'health', 'insurance', 'exchange', 'worse', 'expensive', 'consumer', 'continue', 'apace', 'trump', 'instead', 'could', 'working', 'signing', 'easy', 'making', 'insurance', 'expensive', 'administration', 'chosen', 'otherwise', 'department', 'health', 'human', 'services', 'spend', 'money', 'promotional', 'budget', 'health', 'insurance', 'enrollment', 'campaign', 'criticize', 'affordable']
[]
['gallup']
['request', 'carefirst', 'bigger', 'maryland', 'regulator', 'finish', 'however', 'increase', 'likely', 'smaller', 'company', 'officially', 'seek', 'depend', 'whether', 'maryland', 'newly', 'appoint', 'insurance', 'commissioner', 'republican', 'former', 'insurance', 'broker', 'aggressive', 'predecessor', 'meanwhile', 'maryland', 'consumer', 'counterpart', 'around', 'country', 'choice', 'cigna', 'kaiser', 'permanente', 'unite', 'request', 'lower', 'increase', 'carefirst', 'c

[]
[]
['reconciliation', 'bill', 'simple', 'majority', 'vote', 'republican', 'believe', 'enough', 'vote', 'repeal', 'financial', 'aspect', 'obamacare']
[]
[]
['carson', 'might', 'october', 'speech', 'lewis', 'clarke', 'white', 'mulatto', 'slave', 'woman', 'slavery', 'native', 'kentucky']
[]
[]
['reconciliation', 'rule', 'stipulate', 'provisions', 'direct', 'effect', 'federal', 'budget', 'consideration', 'process', 'could', 'exclude', 'regulatory', 'change', 'conservative', 'republican', 'change', 'rule', 'regard', 'insurance', 'cover', 'rule', 'require', 'legislation', 'reduce', 'budget', 'deficit']
['download']
[]
[]
['house', 'speaker', 'right', 'address', 'question', 'american', 'health', 'house', 'majority', 'leader', 'kevin', 'mccarthy', 'calif', 'tuesday']
['toomey', 'still', 'studying', 'increase', 'premium', 'older', 'american']
[]
[]
['president', 'barack', 'obama', 'thursday', 'ask', 'health', 'insurance', 'company', 'allow', 'individual', 'whose', 'current', 'plan', 'cancel'

['lawyer', 'obama', 'administration', 'weigh', 'behalf', 'transgender', 'student', 'right', 'treat', 'base', 'gender', 'identity', 'gender', 'birth']
['addition', 'enrollment', 'surpass', 'several', 'million', 'people', 'enrol', 'plan', 'second', 'expand', 'coverage']
['instead', 'trump', 'executive', 'order', 'direct', 'federal', 'agency', 'develop', 'rule', 'would', 'allow', 'insurer', 'bypass', 'requirement', 'alternative', 'kind', 'insurance', 'plan']
['right', 'propose', 'surgery', 'center', 'expansion', 'plan', 'imperil', 'republican', 'effort', 'drastically', 'overhaul', 'nation', 'healthcare', 'system', 'leaving', 'state', 'lawmaker', 'choose', 'mcarthur', 'party', 'loyalty', 'press', 'constituent']
['president', 'praise', 'action', 'giving', 'people', 'access', 'plan', 'lower', 'price', 'avoid', 'mention', 'premium', 'already', 'increase', 'game', 'play', 'funding', 'neglect', 'mention', 'association', 'health', 'plan', 'subject', 'state', 'patient', 'protection', 'coverage', 

['remember', 'mccain', 'incidental', 'savior', 'affordable', 'night', 'thumb', 'halt', 'party', 'promise', 'effort', 'overturn', 'major', 'democratic', 'achievement', 'signature', 'achievement', 'democrat', 'become', 'president', 'earn', 'regular', 'bite', 'admonishment', 'president', 'donald', 'trump']
['martin', 'question', 'talking', 'course', 'conversation', 'question', 'going', 'forward']
['expert', 'disagree', 'really', 'obamacare', 'restrain', 'health', 'increase', 'skeptic', 'laszewski', 'health', 'policy', 'strategy', 'associate', 'agree', 'first']
['montagne', 'hamler', 'fugitt', 'executive', 'director', 'association', 'banks', 'thanks', 'joining']
['kliff', 'point', 'certainly', 'pull', 'january', 'trump', 'administration', 'grant', 'city', 'enrollment', 'seeing', 'small', 'amount', 'really', 'enrollment', 'november', 'trump', 'administration', 'letting', 'people', 'enrollment', 'center', 'people', 'staff', 'wait', 'small', 'practical', 'details', 'really', 'terms', 'whether

...

['gaveling', 'continue', 'franken', 'behalf', 'million', 'minnesotan', 'longer', 'discriminate']
['switching', 'federal', 'exchange', 'cause', 'headache', '70,000', 'oregonian', 'sign', 'individual', 'insurance', 'clyde', 'hamstreet', 'cover', 'oregon', 'third', 'chief', 'month']
['republican', 'control', 'congress', 'likely', 'advance', 'legislation', 'amend', 'repeal', 'although', 'likely', 'veto', 'president', 'barack', 'obama']
['antos', 'capretta', 'partial', 'repeal', 'replacement', 'would', 'insurance', 'company', 'obamacare', 'market', 'altogether', 'leaving', 'coverage', 'today', 'insurance']
['flint', 'change', 'already', 'place', 'pipeline', 'increase', 'certainly', 'national', 'health', 'service', 'corps', 'increase', 'number', 'community', 'health', 'center', 'capacity', 'people', 'speaking', 'directly', 'issue', 'reference', 'beginning', 'increase', 'capacity', 'increase', 'training']
['start', 'making', 'doctor', 'appointment']
['ask', 'whether', 'national', 'health', 'r

['likewise', 'democrat', 'shove', 'throat', 'mostly', 'false', 'democrat', 'republican', 'work', 'together', 'pass', 'problem', 'version', 'story', 'republican', 'pushing', 'inaccurate']
['million', 'people', 'insurance', 'policy', 'cancel', 'notification', 'doctor', 'anymore']
['people', 'either', 'choice', 'anyway', 'point', 'point']
['saying', 'truth', 'believing', 'obamacare', 'obama']
[]
['inject', 'epinephrine', 'around', 'since', 'wholesale', 'epinephrine', 'developing', 'world', 'question', 'answer', 'learning', 'pharma', 'price', 'gouge', 'simply', 'business', 'usual']
['question', 'hillary', 'bernie', 'obama', '1,000', 'page', 'legislation', '1,000', 'working', '100,000', 'fighting', 'reform', 'honest', 'talking', 'point']
['state', 'could', 'medicaid', 'expansion', 'leaving', 'million', 'working', 'without', 'coverage', 'burwell']
['money', 'short', 'problem', 'trump', 'could', 'everyone', 'insurance', 'promise']
['point']
['almost', 'certainly', 'expand', 'trump', 'consider

['friday', 'marks', 'final', 'enrollment', 'state', 'although', 'opt', 'extend', 'beyond']
['catch', 'though', 'business', 'actual', 'worker', 'apply', 'business', 'equivalent', 'worker', 'words', 'seasonal', 'employee', 'hours', 'least', 'people']
['remain', 'insurer', 'medica', 'threaten', 'market', 'potentially', 'leaving', 'entire', 'state', 'insurer', 'willing', 'plan', 'individual', 'market', 'medica', 'ultimately', 'decide', 'offer', 'coverage', 'county', 'average', 'premium', 'increase', 'percent']
['things', 'muse', 'senate', 'majority', 'cornyn', 'texas', 'change', 'quickly', 'around']
['brown', 'technical', 'reason', 'relate', 'posture', 'would', 'effort', 'complicate']
['palmer', 'contribute', 'reporting']
['story', 'continue']
['white', 'house', 'aware', 'vote', 'administration', 'official', 'still']
['democratic', 'writer', 'decry', 'giveaway']
['messaging', 'explain', 'people', 'hands', 'daunt', 'process', 'getting', 'insured', 'particularly', 'unnerve', 'people', 'cover

['story', 'continue']
['single', 'meadow', 'front', 'colleague', 'clear', 'message', 'trump', 'want']
['trump', 'meeting', 'philanthropist', 'gates', 'express', 'concern', 'president', 'america', 'first', 'agenda', 'could', 'global', 'development', 'effort', 'desmond', 'hellmann', 'melinda', 'gates', 'foundation', 'blast', 'trump', 'propose', 'budget', 'negative', 'effects', 'safety']
['schumer', 'namby', 'pamby', 'senate', 'majority', 'leader', 'democratic', 'agenda', 'unveil', 'monday', 'voter', 'desert', 'party', 'trump']
['anybody', 'pull', 'underneath', 'going', 'advocate', 'reporter', 'concern', 'write', 'today']
['florida', 'democrat', 'still', 'loser', 'miami', 'herald', 'editorial', 'board']
['brian', 'fitzpatrick', 'freshman', 'buck', 'county', 'lengthy', 'facebook', 'concern', 'first', 'among', 'impact', 'single', 'important', 'issue', 'plague', 'buck', 'montgomery', 'county', 'issue', 'priority', 'congress', 'opioid', 'abuse', 'prevention', 'treatment', 'recovery', 'campaig

['analysis', 'intelligence', 'compare', 'support', 'obamacare', 'republican', 'control', 'district', 'support', 'repeal', 'replace', 'covering', 'broad', 'parameter', 'health']
['orlando', 'first', 'license', 'medical', 'marijuana', 'dispensary', 'open', 'friday', 'orlando', 'sentinel', 'brinkmann']
['would', 'really', 'serious', 'problem', 'senate', 'majority', 'cornyn', 'texas', 'leader', 'party', 'senate', 'campaign', 'committee', 'running', 'campaign', 'since', 'promise', 'repeal', 'replace', 'obamacare', 'imagine', 'supporter', 'would', 'promise']
['casey', 'immigration', 'tweetstorm', 'casey', 'rip', 'trump', 'administration', 'wednesday', 'immigration', 'policy', 'frantically', 'tweet', 'mother', 'deport', 'honduras', 'urging', 'white', 'house', 'intervene', 'hesson', 'seung', 'report', 'furious', 'casey', 'usually', 'placid', 'personality', 'senate', 'reach', 'homeland', 'security', 'secretary', 'kelly', 'thomas', 'homan', 'acting', 'director', 'immigration', 'customs', 'enforc

In [76]:
print(tokens[0:11])

['emanuel', 'kathleen', 'sebelius', 'secretary', 'health', 'human', 'services', 'devise', 'debate', 'implement', 'could']


In [17]:
from gensim import corpora
import pickle

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [18]:
import gensim
#how many topics desired to be created from corpus
num_tops = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = num_tops, id2word=dictionary, passes=7)
#create LDA model and save to call later
ldamodel.save('model.gensim')
#two words that describe each of the five topics
topics = ldamodel.print_topics(num_words=2)
for topic in topics:
    print(topic)

(0, '0.015*"trump" + 0.012*"administration"')
(1, '0.017*"health" + 0.011*"state"')
(2, '0.017*"health" + 0.014*"would"')
(3, '0.020*"health" + 0.012*"insurance"')
(4, '0.014*"insurance" + 0.012*"exchange"')


In [26]:
#creating a dictionary
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model.gensim')

#visualization of topics
import pyLDAvis.gensim
lda_disp = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_disp)
#saliency describes how much that word contributes to the topic
#the distance map maps how closely the topics are related